In [2]:
import pandas as pd
import numpy as np
import torch
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\accstu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
#All sandp500 data, including PrePart, Q&APart, Participants
df_sandp500_ECT = pd.read_csv("D:/Script/sandp500_conference_earning_transcript_participants.csv")

In [29]:
# df_sandp500_ECT['Presentation'][0]
#Copy
df_sandp500_ECT_analysis = df_sandp500_ECT 
#Clean Missing Data
df_sandp500_ECT_cleaned = df_sandp500_ECT_analysis.dropna()
df_sandp500_ECT_cleaned = df_sandp500_ECT_cleaned[df_sandp500_ECT_cleaned.apply(lambda row: row.astype(str).str.contains('No sample').any(), axis=1) == False]
df_sandp500_ECT_cleaned.head()

,title,url,date,ticker,text,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,Presentation,QandA,CompanyParticipants,CallParticipants
0,3M Company (MMM) Q3 2023 Earnings Call Transcript,https://seekingalpha.com/article/4642981-3m-co...,2023-10-24,MMM,3M Company (NYSE:MMM) Q3 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator\nLadies and gentlemen, thank you for ...",Operator\n[Operator Instructions] Our first qu...,"['Bruce Jermeland', 'Michael Roman', 'Monish P...","['Scott Davis', 'Andrew Obin', 'Joe Ritchie', ..."
1,3M Company (MMM) Q2 2023 Earnings Call Transcript,https://seekingalpha.com/article/4619621-3m-co...,2023-07-25,MMM,3M Company (NYSE:MMM) Q2 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator\nLadies and gentlemen, thank you for ...",Operator\n[Operator Instructions] Our first qu...,"['Bruce Jermeland', 'Mike Roman', 'Kevin Rhode...","['Andrew Obin', 'Andrew Kaplowitz', 'Scott Dav..."
2,3M Company (MMM) Q1 2023 Earnings Call Transcript,https://seekingalpha.com/article/4596435-3m-co...,2023-04-25,MMM,3M Company (NYSE:MMM) Q1 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator\nLadies and gentlemen, thank you for ...",Operator\n[Operator Instructions] Our first qu...,"['Bruce Jermeland', 'Mike Roman', 'Monish Pato...","['Andrew Kaplowitz', 'Stephen Tusa', 'Scott Da..."
3,3M Company (MMM) Q4 2022 Earnings Call Transcript,https://seekingalpha.com/article/4571946-3m-co...,2023-01-24,MMM,3M Company (NYSE:MMM) Q4 2022 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator\nLadies and gentlemen, thank you for ...",Operator\n[Operator Instructions] Our first qu...,"['Bruce Jermeland', 'Mike Roman', 'Monish Pato...","['Scott Davis', 'Andrew Kaplowitz', 'Andrew Ob..."
4,3M Company (MMM) Q3 2022 Earnings Call Transcript,https://seekingalpha.com/article/4548991-3m-co...,2022-10-25,MMM,3M Company (NYSE:MMM) Q3 2022 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator\nLadies and gentlemen, thank you for ...",Operator\n[Operator Instructions] Our first qu...,"['Bruce Jermeland', 'Mike Roman', 'Monish Pato...","['Scott Davis', 'Andrew Obin', 'Joe Ritchie', ..."


In [30]:
df_sandp500_ECT_cleaned.dtypes

title                    object
url                      object
date                     object
ticker                   object
text                     object
Symbol                   object
Security                 object
GICS Sector              object
GICS Sub-Industry        object
Headquarters Location    object
Date added               object
CIK                       int64
Founded                  object
Presentation             object
QandA                    object
CompanyParticipants      object
CallParticipants         object
dtype: object

In [3]:
# !pip install transformers
# !pip install nltk
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from nltk import sent_tokenize  

# FinBERT Model
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\accstu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [6]:
#count words
#count for each earning call transcripts Pre and Q&A 字符的长度，包括空格、标点符号等。
df_sandp500_ECT_cleaned['Presentation_char_count']= df_sandp500_ECT_cleaned['Presentation'].str.len()
df_sandp500_ECT_cleaned['QandA_char_count']= df_sandp500_ECT_cleaned['QandA'].str.len()

df_sandp500_ECT_cleaned[['Presentation', 'Presentation_char_count']].head()

,Presentation,Presentation_char_count
0,"Operator\nLadies and gentlemen, thank you for ...",19342
1,"Operator\nLadies and gentlemen, thank you for ...",22477
2,"Operator\nLadies and gentlemen, thank you for ...",24127
3,"Operator\nLadies and gentlemen, thank you for ...",28830
4,"Operator\nLadies and gentlemen, thank you for ...",26719


In [15]:
df_sandp500_ECT_cleaned['Presentation'][0]

"Operator\nLadies and gentlemen, thank you for standing by. Welcome to the 3M Third Quarter Earnings Conference Call. [Operator Instructions] As a reminder, this conference is being recorded, Tuesday, October 24, 2023.\nI would now like to turn the call over to Bruce Jermeland, Senior Vice President of Investor Relations at 3M.\nBruce Jermeland\nThank you and good morning, everyone and welcome to our second quarter earnings conference call. With me today are Mike Roman, 3M's Chairman and Chief Executive Officer; and Monish Patolawala, our President and Chief Financia. Mike and Monish will make some formal comments, and then we will take your questions. Please note that today's earnings release and slide presentation acCompanying this call are posted on the homepage of our Investor Relations website @3m.com.\nPlease turn to Slide two. Please take a moment to read the forward-looking statement. During today's conference call, we'll be making certain predictive statements that reflect our

In [9]:
import re


def extract_and_merge_speeches(text, participants):
    # Initialize a dictionary to keep speeches.
    speeches = {}
    current_speaker = None
    speech_parts = []

    # Split text by newline and process each line.
    lines = text.split('\n')
    for line in lines:
        if line.strip() in participants:
            # If line is a participant's name, process current speech and reset.
            if current_speaker:
                # Join all parts of the current speech and add/merge to speeches dict.
                full_speech = " ".join(speech_parts).strip()
                if current_speaker in speeches:
                    speeches[current_speaker] += " " + full_speech
                else:
                    speeches[current_speaker] = full_speech
            # Reset for the next speaker
            current_speaker = line.strip()
            speech_parts = []
        else:
            # Collect the line into speech parts.
            speech_parts.append(line)

    # Handle the last speaker's speech if any
    if current_speaker and speech_parts:
        full_speech = " ".join(speech_parts).strip()
        if current_speaker in speeches:
            speeches[current_speaker] += " " + full_speech
        else:
            speeches[current_speaker] = full_speech

    return speeches



In [7]:
# # 提取发言的函数
# def extract_speeches(text, participants):
#     speeches = {}
#     current_speaker = None
#     speech_parts = []
#     lines = text.split('\\n')  # 根据您的数据格式调整分割符号
#     for line in lines:
#         # 检查当前行是否包含参与者的名字
#         if any(name in line for name in participants):
#             # 如果当前已经在收集某个参与者的发言，则先保存已收集的内容
#             if current_speaker:
#                 full_speech = ' '.join(speech_parts).strip()
#                 speeches[current_speaker] = speeches.get(current_speaker, '') + full_speech
#             # 重置当前参与者和发言内容
#             current_speaker = line.strip('[]')  # 移除可能的括号
#             speech_parts = []
#         else:
#             # 如果不是新的参与者名字，就将行添加到发言内容中
#             speech_parts.append(line)
#     # 收集最后一个参与者的发言
#     if current_speaker:
#         full_speech = ' '.join(speech_parts).strip()
#         speeches[current_speaker] = speeches.get(current_speaker, '') + full_speech
#     return speeches

In [ ]:
import ast

def safe_literal_eval(item):
    try:
        return ast.literal_eval(item)
    except (ValueError, SyntaxError):
        return []  # 返回一个空列表，或者根据你的需要选择其他默认值

#字符串轉爲list
df_sandp500_ECT_cleaned['CompanyParticipants'] = df_sandp500_ECT_cleaned['CompanyParticipants'].apply(safe_literal_eval)
df_sandp500_ECT_cleaned['CallParticipants'] = df_sandp500_ECT_cleaned['CallParticipants'].apply(safe_literal_eval)
# 输出 CompanyParticipants 和 CallParticipants 列的前5个值
print("CompanyParticipants Samples:")
print(df_sandp500_ECT_cleaned['CompanyParticipants'].head(5))

print("\nCallParticipants Samples:")
print(df_sandp500_ECT_cleaned['CallParticipants'].head(5))

In [36]:

#字符串轉list
df_sandp500_ECT_cleaned['PresentationPreSpeaker'] = df_sandp500_ECT_cleaned.apply(lambda row: extract_and_merge_speeches(row['Presentation'], row['CompanyParticipants']), axis=1)
df_sandp500_ECT_cleaned['QandAPreASpeaker'] = df_sandp500_ECT_cleaned.apply(lambda row: extract_and_merge_speeches(row['QandA'], row['CompanyParticipants']), axis=1)
df_sandp500_ECT_cleaned['QandAPreQAnalyst'] = df_sandp500_ECT_cleaned.apply(lambda row: extract_and_merge_speeches(row['QandA'], row['CallParticipants']), axis=1)

In [39]:
df_sandp500_ECT_cleaned[['PresentationPreSpeaker', 'QandAPreASpeaker','QandAPreQAnalyst']].head(20)

,PresentationPreSpeaker,QandAPreASpeaker,QandAPreQAnalyst
0,{'Bruce Jermeland': 'Thank you and good mornin...,"{'Michael Roman': 'Yes, Scott, I would say the...","{'Scott Davis': 'Good morning, Mike, Monish, a..."
1,{'Bruce Jermeland': 'Thank you and good mornin...,"{'Monish Patolawala': 'Thanks, Andrew. So as I...",{'Andrew Obin': 'Just a question on the outloo...
2,"{'Bruce Jermeland': 'Thank you, and good morni...","{'Mike Roman': 'Good morning. Yes. Andy, as I ...","{'Andrew Kaplowitz': 'Good morning, everyone. ..."
3,"{'Bruce Jermeland': 'Thank you, and good morni...","{'Mike Roman': 'Good morning Scott. Yeah, Scot...","{'Scott Davis': 'Hi, good morning everybody. M..."
4,"{'Bruce Jermeland': 'Thank you, and good morni...","{'Mike Roman': 'Good morning. Sure. Scott, may...","{'Scott Davis': 'Hi. Good morning, Mike and Mo..."
5,{'Bruce Jermeland': 'Thank you and good mornin...,"{'Mike Roman': 'Hey, Andrew. Andrew Obin Yes. ...",{'Andrew Obin': 'Yes. Good morning. Mike Roman...
6,{'Bruce Jermeland': 'Thank you and good mornin...,"{'Mike Roman': 'Good morning, Steve. Yes. Than...","{'Steve Tusa': 'Hey, guys. Good morning. Mike ..."
7,{'Bruce Jermeland': 'Thank you and good mornin...,"{'Monish Patolawala': 'Yes. So Nigel, I would ...",{'Nigel Coe': 'Thanks. Good morning. I wasn’t ...
8,{'Bruce Jermeland': 'Thank you and good mornin...,"{'Monish Patolawala': 'That's great question, ...",{'Nigel Coe': 'Thanks. Good morning and thanks...
9,"{'Bruce Jermeland': 'Thank you, and good morni...","{'Monish Patolawala': 'Yes. Sure, Scott. It's ...","{'Scott Davis': 'Good morning, Mike and Monish..."


In [60]:
from textblob import TextBlob

text = {'Michael Roman': "Yes, Scott,  So I he electrical markets and also automotive aftermarket. So I just wondered any color around those in terms of is it just kind of accelerated destocking distributors just holding off on orders for some reason? Any color at all on auto aftermarket and electrical?"}
# 函数来计算文本的平均极性和主观性
def get_average_sentiment(text):
    sentences = text.split('. ')
    polarities = []
    subjectivities = []
    for sentence in sentences:
        blob = TextBlob(sentence)
        polarities.append(blob.sentiment.polarity)
        subjectivities.append(blob.sentiment.subjectivity)
    avg_polarity = sum(polarities) / len(polarities) if polarities else 0
    avg_subjectivity = sum(subjectivities) / len(subjectivities) if subjectivities else 0
    return avg_polarity, avg_subjectivity

# 应用函数并输出结果
results = {}
for speaker, speech in text.items():
    avg_polarity, avg_subjectivity = get_average_sentiment(speech)
    results[speaker] = {
        'Average Polarity': avg_polarity,
        'Average Subjectivity': avg_subjectivity
    }

print(results)


{'Michael Roman': {'Average Polarity': 0.1350903249620137, 'Average Subjectivity': 0.28342181273472666}, 'Monish Patolawala': {'Average Polarity': 0.1244143410980843, 'Average Subjectivity': 0.3041019643683578}}


In [61]:
df_sandp500_ECT_cleaned.to_csv("D:/Script/有成功分人的講話的.csv", index=False)

In [59]:
#Presentation part Speaker sentiment 
'''01-1 each sentent''' 

# df_sandp500_ECT_cleaned['PresentationPreSpeaker'] = df_sandp500_ECT_cleaned.apply(lambda row: extract_and_merge_speeches(row['Presentation'], row['CompanyParticipants']), axis=1)

df_sandp500_ECT_cleaned['PresentationPreSpeakerSentimentResults'] = None
df_sandp500_ECT_cleaned['PresentationPreSpeakerOverallSentimentScore'] = None
df_sandp500_ECT_cleaned['PresentationPreSpeakerOverallSentimentLabel'] = None


# 第一部分：计算每个句子的情感
for index, row in df_sandp500_ECT_cleaned.iterrows():

    speakers_dict = row['PresentationPreSpeaker']
    all_speakers_sentiments = {}  # 创建一个空字典用于存储所有发言者的情感结果
    # 每个发言者及其发言
    for speaker, text in speakers_dict.items():
        sentences = text.split('. ')
        sentiments = []
        for sentence in sentences:
            result = nlp(sentence)[0]  # FinBERT
            sentiments.append(result)
        all_speakers_sentiments[speaker] = sentiments
    df_sandp500_ECT_cleaned.at[index, 'PresentationPreSpeakerSentimentResults'] = all_speakers_sentiments



In [27]:
# Can change different ways to calculate overall scores
"""
The calculate_overall_sentiment function works as follows:
1.将每个情绪标签的分数乘以对应的数值Multiplies each sentiment label's score with its corresponding numerical value ('Positive': 1, 'Negative': -1, 'Neutral': 0).
2.Averages these weighted scores.
3.Assigns an overall label ('Positive' if average > 0.1, 'Negative' if average < -0.1, 'Neutral' otherwise).。
"""
#EXAMPLE
'''Example:
[
    {'label': 'Positive', 'score': 0.8},
    {'label': 'Neutral', 'score': 0.1},
    {'label': 'Negative', 'score': 0.7}
]

weighted scores:
第一个情绪实例的加权分数是 1 * 0.8 = 0.8
第二个情绪实例的加权分数是 0 * 0.1 = 0
第三个情绪实例的加权分数是 -1 * 0.7 = -0.7
这些加起来得到 0.1，平均分数为 0.1 / 3 ≈ 0.033。因此，整体情绪标签将是 'Neutral'。
'''

sentiment_scores = {'Positive': 1, 'Negative': -1, 'Neutral': 0}

def calculate_overall_sentiment(sentiments):
    weighted_scores = [sentiment_scores[s['label']] * s['score'] for s in sentiments]
    average_score = sum(weighted_scores) / len(weighted_scores) if weighted_scores else 0
    overall_label = 'Neutral'
    if average_score > 0.1:
        overall_label = 'Positive'
    elif average_score < -0.1:
        overall_label = 'Negative'
    return average_score, overall_label

In [62]:
#Presentation part Speaker sentiment 
'''01-2 overall sentence''' 

# overall sentiment result(score and label)
for index, row in df_sandp500_ECT_cleaned.iterrows():
    all_speakers_sentiments = row['PresentationPreSpeakerSentimentResults']
    all_speakers_overall_score = {}
    all_speakers_overall_label = {}
    for speaker, sentiments in all_speakers_sentiments.items():
        overall_score, overall_label = calculate_overall_sentiment(sentiments)
        all_speakers_overall_score[speaker] = overall_score
        all_speakers_overall_label[speaker] = overall_label
    df_sandp500_ECT_cleaned.at[index, 'PresentationPreSpeakerOverallSentimentScore'] = all_speakers_overall_score
    df_sandp500_ECT_cleaned.at[index, 'PresentationPreSpeakerOverallSentimentLabel'] = all_speakers_overall_label


In [63]:
df_sandp500_ECT_cleaned['PresentationPreSpeakerSentimentResults']

0        {'Bruce Jermeland': [{'label': 'Neutral', 'sco...
1        {'Bruce Jermeland': [{'label': 'Neutral', 'sco...
2        {'Bruce Jermeland': [{'label': 'Neutral', 'sco...
3        {'Bruce Jermeland': [{'label': 'Neutral', 'sco...
4        {'Bruce Jermeland': [{'label': 'Neutral', 'sco...
                               ...                        
10188    {'Steve Frank': [{'label': 'Neutral', 'score':...
10189    {'Steve Frank': [{'label': 'Positive', 'score'...
10190    {'Steve Frank': [{'label': 'Neutral', 'score':...
10191                                                   {}
10192                                                   {}
Name: PresentationPreSpeakerSentimentResults, Length: 9926, dtype: object

In [64]:
#QandA part Speaker sentiment 
'''02-1 each sentent'''

# df_sandp500_ECT_cleaned['QandAPreASpeaker'] = df_sandp500_ECT_cleaned.apply(lambda row: extract_and_merge_speeches(row['QandA'], row['CompanyParticipants']), axis=1)

df_sandp500_ECT_cleaned['QandAPreSpeakerSentimentResults'] = None
for index, row in df_sandp500_ECT_cleaned.iterrows():

    speakers_dict = row['QandAPreASpeaker']
    all_speakers_sentiments = {} 
    # 每个发言者及其发言
    for speaker, text in speakers_dict.items():
        sentences = text.split('. ')
        sentiments = []
        for sentence in sentences:
            result = nlp(sentence)[0]  # FinBERT
            sentiments.append(result)
        all_speakers_sentiments[speaker] = sentiments
    df_sandp500_ECT_cleaned.at[index, 'QandAPreSpeakerSentimentResults'] = all_speakers_sentiments


In [65]:
#QandA part Speaker sentiment 
'''02-2 overall sentent'''
df_sandp500_ECT_cleaned['QandAPreSpeakerOverallSentimentScore'] = None
df_sandp500_ECT_cleaned['QandAPreSpeakerOverallSentimentLabel'] = None

for index, row in df_sandp500_ECT_cleaned.iterrows():
    all_speakers_sentiments = row['QandAPreSpeakerSentimentResults']
    all_speakers_overall_score = {}
    all_speakers_overall_label = {}
    for speaker, sentiments in all_speakers_sentiments.items():
        overall_score, overall_label = calculate_overall_sentiment(sentiments)
        all_speakers_overall_score[speaker] = overall_score
        all_speakers_overall_label[speaker] = overall_label
    df_sandp500_ECT_cleaned.at[index, 'QandAPreSpeakerOverallSentimentScore'] = all_speakers_overall_score
    df_sandp500_ECT_cleaned.at[index, 'QandAPreSpeakerOverallSentimentLabel'] = all_speakers_overall_label

In [ ]:
#QandA part Analyst sentiment 
'''03-1 each sentent'''

# df_sandp500_ECT_cleaned['QandAPreQAnalyst'] = df_sandp500_ECT_cleaned.apply(lambda row: extract_and_merge_speeches(row['QandA'], row['CallParticipants']), axis=1)

df_sandp500_ECT_cleaned['QandAPreAnalystSentimentResults'] = None
for index, row in df_sandp500_ECT_cleaned.iterrows():

    speakers_dict = row['QandAPreQAnalyst']
    all_speakers_sentiments = {} 
    # 每个发言者及其发言
    for speaker, text in speakers_dict.items():
        sentences = text.split('. ')
        sentiments = []
        for sentence in sentences:
            result = nlp(sentence)[0]  # FinBERT
            sentiments.append(result)
        all_speakers_sentiments[speaker] = sentiments
    df_sandp500_ECT_cleaned.at[index, 'QandAPreAnalystSentimentResults'] = all_speakers_sentiments

In [33]:
df_sandp500_ECT_cleaned['QandAPreAnalystSentimentResults']

0       {'Scott Davis': [{'label': 'Neutral', 'score':...
1       {'Andrew Obin': [{'label': 'Neutral', 'score':...
2       {'Andrew Kaplowitz': [{'label': 'Positive', 's...
3       {'Scott Davis': [{'label': 'Positive', 'score'...
4       {'Scott Davis': [{'label': 'Neutral', 'score':...
                              ...                        
9921    {'Kevin Ellich': [{'label': 'Neutral', 'score'...
9922    {'Erin Wright': [{'label': 'Neutral', 'score':...
9923    {'Michael Ryskin': [{'label': 'Neutral', 'scor...
9924                                                   {}
9925                                                   {}
Name: QandAPreAnalystSentimentResults, Length: 9926, dtype: object

In [ ]:
#QandA part Analyst sentiment 
'''03-2 overall sentent'''
df_sandp500_ECT_cleaned['QandAPreAnalystOverallSentimentScore'] = None
df_sandp500_ECT_cleaned['QandAPreAnalystrOverallSentimentLabel'] = None

for index, row in df_sandp500_ECT_cleaned.iterrows():
    all_speakers_sentiments = row['QandAPreAnalystSentimentResults']
    all_speakers_overall_score = {}
    all_speakers_overall_label = {}
    for speaker, sentiments in all_speakers_sentiments.items():
        overall_score, overall_label = calculate_overall_sentiment(sentiments)
        all_speakers_overall_score[speaker] = overall_score
        all_speakers_overall_label[speaker] = overall_label
    df_sandp500_ECT_cleaned.at[index, 'QandAPreAnalystOverallSentimentScore'] = all_speakers_overall_score
    df_sandp500_ECT_cleaned.at[index, 'QandAPreAnalystrOverallSentimentLabel'] = all_speakers_overall_label

In [31]:
df_sandp500_ECT_cleaned.to_csv("D:/Script/SanP500withsentiment.csv")